In [8]:
import os
from openai import OpenAI
import pprint

openai_client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [9]:
from datasets import load_dataset

wine_dataset = load_dataset("alfredodeza/wine-ratings")

Found cached dataset csv (/Users/sinanozdemir/.cache/huggingface/datasets/alfredodeza___csv/alfredodeza--wine-ratings-918c1fc1fd00f07a/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
wine_dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'region', 'variety', 'rating', 'notes'],
        num_rows: 32780
    })
    validation: Dataset({
        features: ['name', 'region', 'variety', 'rating', 'notes'],
        num_rows: 200
    })
    test: Dataset({
        features: ['name', 'region', 'variety', 'rating', 'notes'],
        num_rows: 200
    })
})

In [11]:
def convert_wine_to_string(wine):
    description = f'{wine["name"]} is from {wine["region"]} and is a {wine["variety"]}. {wine["notes"]}'
    
    return description

for wine in list(wine_dataset['test'])[:5]:
    print(convert_wine_to_string(wine))
    print()

Jim Barry Lodge Hill Shiraz 2008 is from Clare Valley, South Australia, Australia and is a Red Wine. This wine has deep, dark red/black hues, lifted aromas of mulberry, raspberry, coffee and kirsch. The bramble, chocolate, methol and rosemary fl avours that fi ll the mouth are present in abundance, with delicate, sensual and ripe tannins playing a structural role. This wine is ideally suited to good food, good friends and good conversation.

Argyle Reserve Pinot Noir 2005 is from Willamette Valley, Oregon and is a Red Wine. This Pinot Noir has a seductively rich, super dark ruby-violet color. Aromas leap out of the glass with black cherry, black cherry leather and cedar, along with a ripping portion of cinnamon spice. The wine finishes with a hint of violet flower patch. Use of new oak was delicate, so the aromas are authentic Pinot Noir.  

Cambria Katherine's Vineyard Chardonnay 1997 is from Central Coast, California and is a White Wine. The 1997 Katherines Vineyard Chardonnay is a r

In [12]:
import random

wines = random.sample(list(wine_dataset['test']), 5)

In [13]:
def pretty_print(data, indent=0):
    pp = pprint.PrettyPrinter(indent=indent)
    for item in data:
        for key, value in item.items():
            print(f"{key}: {value}")
        print()
    print()


In [23]:
# Define a dictionary for prices

prices = {  # https://openai.com/pricing
    'gpt-3.5-turbo': {
        'input': 0.001 / 1000,  # per token
        'output': 0.002 / 1000  # per token
    },
    'gpt-4': {
        'input': 0.01 / 1000,  # per token
        'output': 0.03 / 1000  # per token
    }
}


def generate_openai_response(prompt, model='gpt-4', **kwargs):
    pretty_print(prompt)
    chat_completion = openai_client.chat.completions.create(
        messages=prompt,
        model=model,
        **kwargs
    )

    # Calculate the cost
    input_tokens = chat_completion.usage.prompt_tokens
    output_tokens = chat_completion.usage.completion_tokens
    input_cost = input_tokens * prices[model]['input']
    output_cost = output_tokens * prices[model]['output']
    total_cost = input_cost + output_cost
    return chat_completion, {'input_cost': input_cost, 'output_cost': output_cost, 'total_cost': total_cost}

In [24]:
user_description = 'This client hates red wines'

In [25]:
newline = '\n---\n'
res, price = generate_openai_response(
[
    {'role': 'system', 'content': 'You are a wine bot that helps clients understand what kind of wine they want. Given a list of wines and a description of the client. Tell me what wines they want by giving me the only the names of the wines. Include a reason preceding each pick to explain to the user why they might like it.'}, 
    {'role': 'user', 'content': f'User: {user_description}\nWines:\n{f"{newline}".join([convert_wine_to_string(w) for w in wines])}'}
])
    
print(res.choices[0].message.content)

role: system
content: You are a wine bot that helps clients understand what kind of wine they want. Given a list of wines and a description of the client. Tell me what wines they want by giving me the only the names of the wines. Include a reason preceding each pick to explain to the user why they might like it.

role: user
content: User: This client hates red wines
Wines:
L'Ecole 41 Columbia Valley Merlot 2004 is from Columbia Valley, Washington and is a Red Wine. Rich and spicy, with aromatic aromas of nutmeg, clove and cinnamon, this Merlot shows ripe black plum, blackberry and dark flavors encased in a peppery, chocolate, black cherry finish.
---
Armanino The Ringneck Pinot Noir 2012 is from Sonoma Coast, Sonoma County, California and is a Red Wine. This wine is distinguished by stunning beauty and a regal nature. Big, powerful & complex makes this wine truly one of a kind. 
---
Jaffurs Viognier 2015 is from Santa Barbara, Central Coast, California and is a White Wine. This Viognie

In [26]:
price

{'input_cost': 0.005580000000000001,
 'output_cost': 0.00264,
 'total_cost': 0.008220000000000002}

In [27]:
45 / price['total_cost']  # only about 5.5k calls to hit 45 dollars now compared to hosting something on HuggingFace

5474.452554744525

In [29]:
res, price = generate_openai_response(
[
    {'role': 'system', 'content': 'You are a wine bot that helps clients understand what kind of wine they want. Given a list of wines and a description of the client. Tell me what wines they want by giving me the only the names of the wines. Include a reason preceding each pick to explain to the user why they might like it.'}, 
    {'role': 'user', 'content': f'User: {user_description}\nWines:\n{f"{newline}".join([convert_wine_to_string(w) for w in wines])}'}
],
model='gpt-3.5-turbo'
)

45 / price['total_cost']  # you get about 50k calls with gpt 3.5

role: system
content: You are a wine bot that helps clients understand what kind of wine they want. Given a list of wines and a description of the client. Tell me what wines they want by giving me the only the names of the wines. Include a reason preceding each pick to explain to the user why they might like it.

role: user
content: User: This client hates red wines
Wines:
L'Ecole 41 Columbia Valley Merlot 2004 is from Columbia Valley, Washington and is a Red Wine. Rich and spicy, with aromatic aromas of nutmeg, clove and cinnamon, this Merlot shows ripe black plum, blackberry and dark flavors encased in a peppery, chocolate, black cherry finish.
---
Armanino The Ringneck Pinot Noir 2012 is from Sonoma Coast, Sonoma County, California and is a Red Wine. This wine is distinguished by stunning beauty and a regal nature. Big, powerful & complex makes this wine truly one of a kind. 
---
Jaffurs Viognier 2015 is from Santa Barbara, Central Coast, California and is a White Wine. This Viognie

54611.65048543689